# Feature Reduction

### Todo

Add PCA and AutoEncoder funtionality.

How do we deal with outliers

Derived statistics: Sloope, Average, Max/Min

Discuss HVLT correlation/should we use them with andrijana



#### This cell is for defining various OPTIONS used for this notebook (working directory, how many rows and columns pandas displays for a dataframe, etc). 

#### Preferably this cell is also where we do important imports (for example pandas and numpy)

In [393]:
import os 
#Input the directory where your joined_data.csv is located 
#os.chdir('C:/Users/Trond/Documents/Master 2020/Processed data')
os.chdir('C:/Users/Briggstone/Documents/Master 2020/Processed data')
# os.chdir('C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data')

#Where you want the csv file of the merged data to be placed
output_filepath = 'C:/Users/Briggstone/Documents/Master 2020/Processed data'
#output_filepath = 'C:/Users/Trond/Documents/Master 2020/Processed data'
#output_filepath = 'C:/Users/MyPC/Documents/Andrijana/UiS/DATMAS Master oppgave/Processed data'

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.model_selection import train_test_split

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)


# Joined imputed data to import, 0 = MODE/MEAN IMPUTATION, 1 = SIMILARITY MEASURE
MV_FLAG = 0

#The portion of data in the test set
TEST_PORTION = 0.2


#Various flags for which preprocessing steps to take and what graphics we want to show:

# 0 = Do not show the distributions, 1 = Show the distributions
SHOW_DISTRIBUTIONS = 0

# 0 = no derived total scores, 1 = derived total scores with no dichotomization, 2 = derived total scores with dichotomization
''' 0 not implemented pending review ''' 
DERIVED_TOTAL_SCORES = 1

# 0 = no dimensionality reduction, 1 = PCA 
DIMENSIONALITY_REDUCTION = 1

if DIMENSIONALITY_REDUCTION != 0:
    #Set ratio of variance to keep, translates to the number of dimensions of PCA kept
    VARIANCE_KEEP = 0.9


#### In this cell we import our joined imputed data based on MV_FLAG

In [394]:
if MV_FLAG == 0:
    data = pd.read_csv('joined_data_mm.csv') # missing values filled with mean/median
else: 
    data = pd.read_csv('joined_data_heom.csv') # missing values filled based on HEOM measure



#### In this cell we define a function that shows us interesting distributions for our response

In [395]:
def show_dist (data):
    data_HALL_AFTER = data.copy(deep = True)
    unique_patno = data.PATNO.unique()
    event_ids = np.sort(data.EVENT_ID.unique())
    for x in unique_patno:
        subject_data = data_HALL_AFTER.loc[data_HALL_AFTER.PATNO == x, ["EVENT_ID", "HALL"]]
        subject_event_ids = np.sort(subject_data.EVENT_ID.unique())
        first_hall = 0
        for i in subject_event_ids:
            if first_hall != 1 and subject_data.loc[subject_data.EVENT_ID == i, ["HALL"]].values[0] == 1:
                    first_hall = 1
            elif first_hall == 1:
                    data_HALL_AFTER.loc[(data_HALL_AFTER.PATNO == x) & (data_HALL_AFTER.EVENT_ID == i), ["HALL"]] = 1

    event_id_to_years = {
        "BL" : "Baseline",
        "V04": "1-Year",
        "V06": "2-Year",
        "V08": "3-Year",
        "V10": "4-Year",
        "V12": "5-Year",
        "V13": "6-Year",
        "V14": "7-Year",
        "V15": "8-Year"   
    }

    '''
    distribution1 = Total subjects in the study per year
    distribution2 = Subjects with a non-missing datapoint per year
    distribution3 = Percentage of remaining subjects in the study with a non-missing datapoint per year
    distribution4 = Number of subjects with a non-missing datapoint that report hallucinations per year
    distribution5 = Number of subjects with a non-missing datapoint that report hallucinations or have reported hallucinations a previous years per year
    distribution6 = Percentage of subjects with a non-missing datapoint that report hallucinations per year
    distribution7 = Percentage of subjects with a non-missing datapoint that report hallucinations or have reported hallucinations previous years per year
    distribution8 = Number of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    distribution9 = Percentage of previously non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    '''

    #Total subjects in the study per year
    distribution1 = {}

    subjects = data.PATNO.unique().size
    for x in event_ids:
        distribution1 [event_id_to_years[x]] = subjects
        subjects -= data.loc[data.LAST_EVENT_ID == x, "PATNO"].unique().size


    #Subjects with a datapoint per year
    distribution2 = {}

    for x in event_ids:
        distribution2 [event_id_to_years[x]] = data.loc[data.EVENT_ID == x, "PATNO"].unique().size



    #Percentage of remaining subjects with a datapoint per year       
    distribution3 = {}
    subjects = data.PATNO.unique().size
    for x in event_ids:
        distribution3 [event_id_to_years[x]] = (data.loc[data.EVENT_ID == x, "PATNO"].unique().size / subjects) * 100
        subjects -= data.loc[data.LAST_EVENT_ID == x, "PATNO"].unique().size


    # Number of subjects with a datapoint that report hallucinations per year
    distribution4 = {}
    for x in event_ids:
        subjects_hall = data.loc[(data.EVENT_ID == x) & (data.HALL == 1), "PATNO"].values
        distribution4[event_id_to_years[x]] = subjects_hall.size 


    #Number of subjects with a datapoint that report hallucinations or have reported hallucinations priorly per year
    distribution5 = {}

    for x in event_ids:
        subjects_hall = data_HALL_AFTER.loc[(data_HALL_AFTER.EVENT_ID == x) & (data_HALL_AFTER.HALL == 1), "PATNO"].values
        distribution5[event_id_to_years[x]] = subjects_hall.size 


    #Percentage of subjects with a non-missing datapoint that report hallucinations per year
    distribution6 = {}

    for x in event_ids:
        subjects_hall = data.loc[(data.EVENT_ID == x) & (data.HALL == 1), "PATNO"].values
        distribution6[event_id_to_years[x]] = (subjects_hall.size / data.loc[data.EVENT_ID == x, "PATNO"].unique().size) * 100

    #Percentage of subjects with a non-missing datapoint that report hallucinations or have reported hallucinations previous years per year
    distribution7 = {}

    for x in event_ids:
        subjects_hall = data_HALL_AFTER.loc[(data_HALL_AFTER.EVENT_ID == x) & (data_HALL_AFTER.HALL == 1), "PATNO"].values
        distribution7[event_id_to_years[x]] = (subjects_hall.size / data.loc[data.EVENT_ID == x, "PATNO"].unique().size) * 100

    #Number of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    distribution8 = {}

    tempdata = data_HALL_AFTER.copy(deep = True)
    for x in event_ids: 
        subjects_hall = tempdata.loc[(tempdata.EVENT_ID == x) & (tempdata.HALL == 1), "PATNO"].values
        distribution8[event_id_to_years[x]] = subjects_hall.size

        for i in subjects_hall:
            tempdata = tempdata[tempdata.PATNO != i]

    #Percentage of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    distribution9 = {}

    tempdata = data_HALL_AFTER.copy(deep = True)
    for x in event_ids: 
        subjects_hall = tempdata.loc[(tempdata.EVENT_ID == x) & (tempdata.HALL == 1), "PATNO"].values
        distribution9[event_id_to_years[x]] = (subjects_hall.size / data.loc[data.EVENT_ID == x, "PATNO"].unique().size) * 100

        for i in subjects_hall:
            tempdata = tempdata[tempdata.PATNO != i]






    fig, axs = plt.subplots(5, 2)
    fig.set_size_inches(25, 25)
    fig.tight_layout(pad=5.0)
    fig.delaxes(axs[4,1])


    # distribution1 = Total subjects in the study per year
    axs[0,0].bar(distribution1.keys(), distribution1.values(), width=.5, color='g')
    axs[0,0].title.set_text("Total subjects in the study per year")

    #distribution2 = Subjects with a non-missing datapoint per year
    axs[0,1].bar(distribution2.keys(), distribution2.values(), width=.5, color='r')
    axs[0,1].title.set_text("Subjects with a non-missing datapoint per year")

    #distribution3 = Percentage of remaining subjects in the study with a non-missing datapoint per year
    axs[1,0].bar(distribution3.keys(), distribution3.values(), width=.5, color='y')
    axs[1,0].title.set_text("Percentage of remaining subjects in the study with a non-missing datapoint per year")

    #distribution4 = Number of subjects with a non-missing datapoint that report hallucinations per year
    axs[1,1].set_ylim(0,70)
    axs[1,1].bar(distribution4.keys(), distribution4.values(), width=.5, color='k')
    axs[1,1].title.set_text("Number of subjects with a non-missing datapoint that report hallucinations per year")

    #distribution5 = Number of subjects with non-missing datapoint that report hallucinations \n or have reported hallucinations a previous year per year
    axs[2,0].set_ylim(0,70)
    axs[2,0].bar(distribution5.keys(), distribution5.values(), width=.5, color='c')
    axs[2,0].title.set_text("Number of subjects with non-missing datapoint that report hallucinations \n or have reported hallucinations a previous year per year")

    #distribution6 = Percentage of subjects with a non-missing datapoint that report hallucinations per year
    axs[2,1].set_ylim(0,50)
    axs[2,1].bar(distribution6.keys(), distribution6.values(), width=.5, color='m')
    axs[2,1].title.set_text("Percentage of subjects with a non-missing datapoint that report hallucinations per year")

    #distribution7 = Percentage of subjects with a non-missing datapoint that report hallucinations or have reported hallucinations a previous year per year
    axs[3,0].set_ylim(0,50)
    axs[3,0].bar(distribution7.keys(), distribution7.values(), width=.5, color= '#3933FF')
    axs[3,0].title.set_text("Percentage of subjects with a non-missing datapoint that report hallucinations \n or have reported hallucinations a previous year per year")

    #Number of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    axs[3,1].set_ylim(0,30)
    axs[3,1].bar(distribution8.keys(), distribution8.values(), width=.5, color= '#EE910C')
    axs[3,1].title.set_text("Number of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year")

    #Percentage of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year
    axs[4,0].set_ylim(0,20)
    axs[4,0].bar(distribution9.keys(), distribution9.values(), width=.5, color= '#978A8A')
    axs[4,0].title.set_text("Percentage of previoussly non-hallucinating subjects with a non-missing datapoint that present as new hallucinations cases per year")


#### In this cell we dichtomize our response and look at various distributions if flag is set

In [396]:
'''Dihotomize NP1HALL dependent variable'''

# if the patient has not suffered hallucinations, we consider it 0
# if the patient has suffered >= 1 times hallucinations, we consider it 1   
data['HALL'] = data['NP1HALL'].apply(lambda x: np.where(x >=1, 1, 0))

data.drop('NP1HALL', inplace = True, axis = 1)
 
if SHOW_DISTRIBUTIONS == 1:
    show_dist(data)


#### In this cell we define the functions we need to calculate derived values from various variables

In [397]:
#Functions for calculating derived values for various data tables

def hvlttot (df):
    
    component_vars = ["HVLTRT1", "HVLTRT2", "HVLTRT3", "HVLTREC", "HVLTFPRL", "HVLTFPUN"]
    
    #Immediate/Total recall
    df["HVLT_TOTAL"] = df.loc[:,["HVLTRT1", "HVLTRT2", "HVLTRT3"]].sum(axis = 1)
    
    #Discrimination Recognition
    df["HVLT_DISCRIM"] = df.HVLTREC - (df.HVLTFPRL + df.HVLTFPUN)
    
    #Retention
    df["HVLT_RETENTION"] = df.HVLTRDLY / df.loc[:,["HVLTRT2", "HVLTRT3"]].max(axis = 1)
    
    df.drop(component_vars,inplace = True, axis = 1)
    
    
def mcatot (df):
    ''' Montreal cognitive test
    sum variables and 1 point to score if education years <= 12 and score < 30
    '''
    
    component_vars = ["MCAALTTM", "MCACUBE", "MCACLCKC", "MCACLCKN", "MCACLCKH", "MCALION", "MCARHINO", "MCACAMEL", \
    "MCAFDS", "MCABDS", "MCAVIGIL", "MCASER7", "MCASNTNC", "MCAVF", "MCAABSTR", "MCAREC1", "MCAREC2", "MCAREC3", "MCAREC4", \
    "MCAREC5", "MCADATE", "MCAMONTH", "MCAYR", "MCADAY", "MCAPLACE", "MCACITY"]
    
    df['MCATOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    
    df['MCATOT'] = df.apply(lambda row_wise: row_wise["MCATOT"] + 1 if row_wise["EDUCYRS"] <= 12 and row_wise["MCATOT"] < 30 else row_wise["MCATOT"] , axis=1)   
    

def vlttot (df):
    ''' Semantic Fluency
    VLTANIM, VLTVEG,VLTFRUIT need to be summed in order to obtain a final score'''
    
    component_vars = ["VLTANIM", "VLTVEG", "VLTFRUIT"]
    
    df['VLTTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

    
def remqtot (df):
    '''REM sleep behavior disorder (RBD)'''
    
    component_vars = ["STROKE","HETRA", "PARKISM", "RLS", "NARCLPSY", "DEPRS", "EPILEPSY", "BRNINFM", "CNSOTH"]
        
    score = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    
    # 1 point if any of these component variables had a 1, else 0
    score = pd.Series(np.where(score >= 1, 1, 0))
    
    df.drop(component_vars, inplace = True, axis = 1)

    component_vars = ["DRMVIVID", "DRMAGRAC", "DRMNOCTB", "SLPLMBMV", "SLPINJUR", \
                      "DRMVERBL", "DRMFIGHT", "DRMUMV", "DRMOBJFL", "MVAWAKEN", "DRMREMEM", "SLPDSTRB"]
    
    score += df.loc[:, component_vars].sum(axis = 1, skipna = False)

    df['REMTOT'] = score
    df.drop(component_vars, inplace = True, axis = 1)

    
def gdsstot (df):
    '''Geriatric Depression Scale'''
    
    component_vars = ["GDSSATIS", "GDSDROPD", \
    "GDSEMPTY", "GDSBORED", "GDSGSPIR", "GDSAFRAD", "GDSHAPPY", "GDSHLPLS", "GDSHOME", "GDSMEMRY", "GDSALIVE", "GDSWRTLS", "GDSENRGY", \
    "GDSHOPLS", "GDSBETER"]
    
    df['GDSSTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

    
def sidttot (df):
    '''Olfactory impairment: University of Pennsylvania Smell ID Test'''
    component_vars = ["UPSITBK1", "UPSITBK2", "UPSITBK3", "UPSITBK4"]
    
    df['SIDTTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

    
def epsstot (df):
    '''Epworth Sleepiness Scale'''
    
    component_vars = ["ESS1", "ESS2", \
    "ESS3", "ESS4", "ESS5", "ESS6", "ESS7", "ESS8"]
    
    df['EPSSTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

    
def scoptot(df):
    '''Scales for Outcomes in Parkinson’s Disease–Autonomic'''

    component_vars = ["SCAU1", "SCAU2", \
    "SCAU3", "SCAU4", "SCAU5", "SCAU6", "SCAU7", "SCAU8", "SCAU9", "SCAU10", "SCAU11", "SCAU12", "SCAU13", \
    "SCAU14", "SCAU15", "SCAU16", "SCAU17", "SCAU18", "SCAU19", "SCAU20", "SCAU21", "SCAU22", "SCAU23", "SCAU24", "SCAU25"]
    
    df['SCOPTOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)
    

def msu3tot(df):
    '''Movement Disorders Society–Unified Parkinson Disease Rating Scale'''
    
    component_vars = ['NP3BRADY', 'NP3FACXP', 'NP3FRZGT', \
    'NP3FTAPL', 'NP3FTAPR', 'NP3GAIT', 'NP3HMOVL', 'NP3HMOVR', 'NP3KTRML', 'NP3KTRMR', 'NP3LGAGL', 'NP3POSTR', 'NP3PRSPL', 'NP3PRSPR', \
    'NP3PSTBL', 'NP3PTRML', 'NP3PTRMR', 'NP3RIGLL', 'PN3RIGRL', 'NP3RIGN', 'NP3RIGRU', 'NP3RISNG', 'NP3RTALJ', 'NP3RTALL', 'NP3RTALU', \
    'NP3RTARL', 'NP3RTARU', 'NP3RTCON', 'NP3SPCH', 'NP3TTAPL', 'NP3TTAPR', 'NP3LGAGR', 'NP3RIGLU']
       
    df['MSU3TOT'] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    #df.drop(component_vars, inplace = True, axis = 1) #cannot drop, variables needed in tremor and pigd

    
def tremor(df):
    '''Tremor score'''
    
    component_vars = ["NP2TRMR", "NP3PTRMR", "NP3PTRML", "NP3KTRMR", "NP3KTRML", "NP3RTARU", "NP3RTALU", "NP3RTARL", "NP3RTALL", \
    "NP3RTALJ", "NP3RTCON"]
    
    df['TREMOR'] = df.loc[:, component_vars].mean(axis = 1, skipna = False)
    #df.drop(component_vars, inplace = True, axis = 1) #cannot drop, variables needed in tremor and pigd
    
    
def pigd(df):
    '''PIGD score'''
    
    component_vars = ["NP2WALK", "NP2FREZ", "NP3GAIT", "NP3FRZGT", "NP3PSTBL"]
    df['PIGD'] = df.loc[:, component_vars].mean(axis = 1, skipna = False)
    
    component_vars = ['NP3BRADY', 'NP3FACXP', 'NP3FRZGT', \
    'NP3FTAPL', 'NP3FTAPR', 'NP3GAIT', 'NP3HMOVL', 'NP3HMOVR', 'NP3KTRML', 'NP3KTRMR', 'NP3LGAGL', 'NP3POSTR', 'NP3PRSPL', 'NP3PRSPR', \
    'NP3PSTBL', 'NP3PTRML', 'NP3PTRMR', 'NP3RIGLL', 'PN3RIGRL', 'NP3RIGN', 'NP3RIGRU', 'NP3RISNG', 'NP3RTALJ', 'NP3RTALL', 'NP3RTALU', \
    'NP3RTARL', 'NP3RTARU', 'NP3RTCON', 'NP3SPCH', 'NP3TTAPL', 'NP3TTAPR', "NP2TRMR", "NP2WALK", "NP2FREZ", 'NP3LGAGR', 'NP3RIGLU']
    #cannot drop before we discuss missing values
    df.drop(component_vars, inplace = True, axis = 1) #drop everything from msu3tot, tremor and pig
    
def famtot (df):
    ''' Raw sum of PD family history'''
    component_vars = ["BIOMOMPD", "BIODADPD", "FULSIBPD", "HAFSIBPD", "MAGPARPD", "PAGPARPD", "MATAUPD", "PATAUPD", "KIDSPD"]   
    df["FAMTOT"] = df.loc[:, component_vars].sum(axis = 1, skipna = False)
    df.drop(component_vars, inplace = True, axis = 1)

#### In this cell we define functions for encoding and dichotomizing variables

In [398]:
#Functions for encoding and dichotomizing variables

def ratio(x, y):
    ''' Calculate TD/PGID ratio'''
    
    if y == 0:
        if x == 0:
            ratio = 0 #indeterminate
        else: 
            ratio =1 #TD
    elif x/y >= 1.15:
        ratio = 1 #TD
    elif x/y <= 0.9:
        ratio = 2 #PIGD
    else:
        ratio = 0 #indeterminate 
    return ratio

def td_pigd_classification(df):
    '''Tremor/PIGD ratio'''
    
    component_vars = ['TREMOR', 'PIGD']
    df['TD_PIGD_RATIO'] = df.apply(lambda x: ratio(x['TREMOR'], x['PIGD']), axis=1)
    df.drop(component_vars, inplace = True, axis = 1)
   

def famhist(df):
    '''Family history of Parkinson's Disease'''
    
        
    score = df.FAMTOT
    
    # if score >= 1 then 1, else 0
    # if score = NaN, then 0
    score = pd.Series(np.where(score >= 1, 1, 0))
    
    df.drop(["FAMTOT"], inplace = True, axis = 1)
    df['FAMHIST'] = score
    
    
def sleepy(df):
    '''Dichotomize EPSSTOT, Epworth Sleepiness Scale'''
    
    # if score < 10 subjects will be classified as 0 (not sleepy)
    # if score >= 10 subject will be classified as 1 (sleepy).
    df['SLEEPY'] = df['EPSSTOT'].apply(lambda x: np.where(x >=10, 1, 0))

    df.drop('EPSSTOT', inplace = True, axis = 1)


def depr(df):
    '''Dichotomize GDSSTOT, Geriatric Depression Scale'''
    
    # if score <5 subjects will be classified as 0 (non-depressed).
    # if score >= 5 subjects will be classified as 1 (depressed) 
    df['DEPR'] = df['GDSSTOT'].apply(lambda x: np.where(x >=5, 1, 0))

    df.drop('GDSSTOT', inplace = True, axis = 1)


def rbd(df):
    '''Dichotomize REMTOT, REM sleep behavior disorder (RBD)'''
       
    # if score <5 subjects will be classified as 0 (RBD negative).
    # if score >= 5 subjects will be classified as 1 (RBD positive) 
    df['RBD'] = df['REMTOT'].apply(lambda x: np.where(x >=5, 1, 0))

    df.drop('REMTOT', inplace = True, axis = 1)

#### Derived_values and dichotomization wrapper function

In [399]:
def derived_values(df, dichotomize):
    '''All derived_values preprocessing together'''
    hvlttot(df)
    mcatot(df)
    vlttot(df)
    remqtot(df)
    gdsstot(df)
    sidttot(df)
    epsstot(df)
    scoptot(df)
    msu3tot(df)
    tremor(df)
    pigd(df)
    famtot(df)
    
    if dichotomize == True:
        famhist(df)
        sleepy(df)
        depr(df)
        rbd(df)  
        td_pigd_classification(df)

#### In this cell we define our functions for dimensionality reduction

#### In this cell we define our function for standardizing numerical and ordinal variables and one-hot encoding categorical ones

In [400]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

def PD_MED_USE_ONEHOT (PD_SERIES):
    
    df = pd.DataFrame()
    df["PD_MED_USE"] = PD_SERIES
    df["MED_NO"] = 0
    df["MED_LEV"] = 0
    df["MED_AG"] = 0
    df["MED_OTHER"] = 0
    
    for index, row in df.iterrows():
        value = row["PD_MED_USE"]
        if value == 0:
            df.at[index,"MED_NO"] = 1
        if value == 1:
            df.at[index,"MED_LEV"] = 1
        elif value == 2:
            df.at[index,"MED_AG"] = 1
        elif value == 3:
            df.at[index,"MED_OTHER"] = 1
        elif value == 4: 
            df.at[index,"MED_LEV"] = 1
            df.at[index,"MED_OTHER"] = 1
        elif value == 5:
            df.at[index,"MED_LEV"] = 1
            df.at[index,"MED_AG"] = 1    
        elif value == 6:
            df.at[index,"MED_AG"] = 1
            df.at[index,"MED_OTHER"] = 1  
        elif value == 7:
            df.at[index,"MED_LEV"] = 1
            df.at[index,"MED_AG"] = 1   
            df.at[index,"MED_OTHER"] = 1 
    df.drop("PD_MED_USE", inplace = True, axis = 1)
    return df

    
def standardize (numeric_train, numeric_test):
    
    scaler = StandardScaler()
    scaler.fit(numeric_train)
    
    numeric_train[numeric_train.columns] = scaler.transform(numeric_train[numeric_train.columns])
    numeric_test[numeric_test.columns] = scaler.transform(numeric_test[numeric_test.columns])
    
    
    return numeric_train,numeric_test

def one_hot_encode_helper (df, column_types):
    
    columns = df.columns
    tempdf = pd.DataFrame()
    for x in columns:
        column_info = column_types.loc[column_types.COLUMN_NAME == x,:]
        if column_info["0"].values[0] == "NO_USE":
            tempdf[x] = df[x]
        elif column_info["0"].values[0] == "USE_SPECIAL_FUNCTION":
            code_string = x + "_ONEHOT"
            data_return = globals()[code_string](df[x])
            for x in data_return.columns:
                tempdf[x] = data_return[x]
        else:
            categorical_predictor = np.array(df[x]).reshape(-1,1)
            enc = OneHotEncoder(sparse = False)
            enc.fit(categorical_predictor)
            categorical_predictor = enc.transform(categorical_predictor)
            
            for i in range(0,categorical_predictor.shape[1]):
                column_name = column_types.loc[column_types.COLUMN_NAME == x, str(i)].values[0]
                tempdf[column_name] = categorical_predictor[:,i]
                
    return tempdf
            
def one_hot_encode(categorical_train, categorical_test, column_types):
    
    categorical_train = one_hot_encode_helper(categorical_train, column_types)
    categorical_test = one_hot_encode_helper(categorical_test, column_types)
    
    return categorical_train, categorical_test

def standardize_and_encode (train, test, data_types_filename):
    
    response_train = train.pop("HALL")
    response_test = test.pop("HALL")
    
    column_types = pd.read_csv(data_types_filename)
    numeric_columns = column_types.loc[column_types.DATA_TYPE != "Categorical", "COLUMN_NAME"].values 
    numeric_columns = np.intersect1d(numeric_columns, train.columns.values)   
    
    
    numeric_train = train.loc[:, numeric_columns]
    numeric_test = test.loc[:, numeric_columns]
    
    categorical_train = train.drop(numeric_columns, axis = 1)
    categorical_test = test.drop(numeric_columns, axis = 1)    
    
    numeric_train,numeric_test = standardize(numeric_train,numeric_test)
    categorical_train,categorical_test = one_hot_encode(categorical_train, categorical_test, column_types)
    
    train = pd.concat([numeric_train, categorical_train], axis = 1)
    train = pd.concat([train,response_train], axis = 1)
    
    test = pd.concat([numeric_test, categorical_test], axis = 1)  
    test = pd.concat([test,response_test], axis = 1)
    
    return train,test

#### In this cell we define our functions for dimensionality reduction

In [401]:
from sklearn.decomposition import PCA

def dimreduc_PCA (train, test, variance_keep):
    non_predictor_variables = ["HALL", "EVENT_ID", "LAST_EVENT_ID", "PATNO"]
    temptrain = train.loc[:, non_predictor_variables]
    temptest = test.loc[:, non_predictor_variables]
    train.drop(non_predictor_variables, axis = 1, inplace = True)
    test.drop(non_predictor_variables, axis = 1, inplace = True)
    
    pca = PCA(svd_solver = "full")
    pca.fit(train)
    
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    if variance_keep == 1:
        index = np.where(cumsum > 0)[0][-1]
    else:    
        index = np.where(cumsum >= variance_keep)[0][0]
    
    train_transform = pca.transform(train)
    test_transform = pca.transform(test)
    
    for i in range(0,index + 1):
        temptrain["PCA" + str(i)] = train_transform[:,i]
        temptest["PCA" + str(i)] = test_transform[:,i]
        
    return temptrain,temptest
    
    

#### In this cell we define a function that prints our a correlation heatmap of our features

In [402]:
def corr_heatmap (df):
    #Using Pearson Correlation
    non_relevant_features = ["PATNO", "EVENT_ID", "LAST_EVENT_ID"]
    data = df.drop(non_relevant_features, axis = 1)
    plt.figure(figsize=(12,10))
    cor = data.corr()
    sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
    plt.show()

#### In this cell we define our train test split function

In [403]:
'''
We split into training and test so that we base standardization or dimensionality reduction mapping on training set only
We have to ensure that all longitudinal observations from one patient ends up in the same sample, we therefore split on PATNOs
We stratify our sampling by converting our response to HALL_EVER. 
This ensures an equal distribution of subjects who never experience hallucinations and those who do in both training and test
Both longitudinal and non-longitudinal models predict on HALL_EVER, although in slightly different forms 
'''
def split_data (data):
    tempdata = pd.DataFrame(data.PATNO.unique(), columns = ["PATNO"])

    HALL_EVER = []
    for id in data.PATNO.unique():

        if data.loc[(data.PATNO == id) & (data.HALL == 1), "HALL"].empty:
            HALL_EVER.append(0)
        else:
            HALL_EVER.append(1)

    Y = HALL_EVER
    X = tempdata

    train, test, _, _ = train_test_split( X, Y, test_size= TEST_PORTION, random_state= 1, stratify= Y)

    train = data.merge(train, how = "inner", on = "PATNO")
    test = data.merge(test, how = "inner", on = "PATNO")
    
    return train,test

#### In this cell we do the final preprocessing as dictated by the flags set

In [404]:
#We split our data into training and test sets
train,test = split_data(data)

#Derived Scores option:
if DERIVED_TOTAL_SCORES != 0:
    if DERIVED_TOTAL_SCORES == 1: 
        derived_values(train, False)
        derived_values(test, False) 
    elif DERIVED_TOTAL_SCORES == 2:
        derived_values(train, True)
        derived_values(test, True) 
    train,test = standardize_and_encode(train,test,"Column_Data_Types_Final.csv")    
else:
    standardize_and_encode(train,test,"Column_Data_Types.csv")   
    
#Dimensionality reduction option:
if DIMENSIONALITY_REDUCTION != 0:
    if DIMENSIONALITY_REDUCTION == 1:
        train,test = dimreduc_PCA(train,test, VARIANCE_KEEP)
        
        

train.to_csv(output_filepath + '/train.csv', index = False)


In [405]:
train.head()

,HALL,EVENT_ID,LAST_EVENT_ID,PATNO,PCA0,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,PCA11,PCA12,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18
0,0,BL,V15,3400,-2.188641,0.887098,-2.295754,-0.756353,0.141888,0.388110,-0.825700,-0.713504,0.947360,1.920573,0.563555,1.072937,-0.140934,-0.621669,-0.169499,-0.112747,-0.402626,1.185670,0.798751
1,0,V04,V15,3400,-0.873949,3.138870,-1.657954,-2.125143,0.340775,2.102233,-0.590161,1.174360,-0.278231,1.316030,-0.327101,1.343914,-0.095577,-0.937931,-0.366745,-0.868640,-0.239687,-0.344757,0.665022
2,0,V06,V15,3400,-1.101498,1.967299,-1.017423,-1.497440,0.401893,1.515011,-0.375514,0.583075,-0.030843,1.204751,-0.086451,-0.029916,-0.016710,-0.015810,-0.238003,-0.585258,-0.636017,-0.456883,1.214170
3,0,V12,V15,3400,-0.440483,2.004255,-2.402404,-1.806546,-0.109791,0.020855,0.534478,0.735592,-0.099293,2.170940,0.601853,-0.136230,-0.196459,-0.012577,-0.975327,0.599568,0.077437,-1.334188,1.673583
4,0,V08,V15,3400,-0.950769,4.199369,-2.595666,-1.375320,0.408760,2.578525,-0.110963,0.164575,-0.225284,1.572659,0.039217,-0.444123,0.008127,-0.529255,0.468459,-0.693360,0.349033,-0.336908,0.891727
